# Рынок BMW

## Описание

Для простоты рассмотрим рынок BMW, выпущенных после 2010 года. Данные берутся с 3 наиболее популярных сайтов по продаже авто в России — Авто.ру, Авито, Дром

## Сбор данных

Так как каждый сайт имеет свой HTML-код, то код для парсинга каждого сайта будет отличаться. Используется динамический парсинг с помощью Selenium

### Основные критерии

- Модель
- Год выпуска
- Объем двигателя
- Бензин/дизель
- Привод
- Мощность (л.с.)
- Пробег
- КПП
- Цена
- Тип (седан/внедорожник/кроссовер) (???)


In [140]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import time
import re
import drom_parser

### Парсинг с Дрома

In [146]:
num_pages = 100                      # кол-во страниц, которые хоти обработать
year = 2010                         # самый ранний год выпуска машины
non_dymanic = True                  # если нужно, чтобы браузер НЕ открывался в явном виде, то поставить True

In [147]:
htmls = drom_parser.parse_html(year, num_pages, non_dymanic)

In [157]:
cars = drom_parser.collect_data(htmls)

In [158]:
for el in cars:
    print(el)

BMW X3, 2024 | X3 | 2024 | 50 |2.0L | 245 HP | бензин | 4WD | АКПП | 7600000 ₽ | https://auto.drom.ru/moscow-area/bmw/x3/561072744.html
BMW X2, 2019 | X2 | 2019 | 21 400 |1.5L | 140 HP | бензин | передний | робот | 2540000 ₽ | https://auto.drom.ru/reutov/bmw/x2/423782337.html
BMW X5, 2011 | X5 | 2011 | 198 020 |3.0L | 306 HP | бензин | 4WD | АКПП | 2030000 ₽ | https://auto.drom.ru/usk-kut/bmw/x5/244354770.html
BMW X5, 2019 | X5 | 2019 | 69 950 |3.0L | 340 HP | бензин | 4WD | АКПП | 7295000 ₽ | https://auto.drom.ru/vladivostok/bmw/x5/174962040.html
BMW X5, 2024 | X5 | 2024 | 4 993 |4.4L | 530 HP | бензин | 4WD | АКПП | 16800000 ₽ | https://auto.drom.ru/moscow/bmw/x5/814907961.html
BMW X5, 2023 | X5 | 2023 | 7 226 |4.4L | 625 HP | бензин | 4WD | АКПП | 21000000 ₽ | https://auto.drom.ru/moscow/bmw/x5/101621077.html
BMW X5, 2024 | X5 | 2024 | 118 |3.0L | 381 HP | бензин | 4WD | АКПП | 12700000 ₽ | https://auto.drom.ru/moscow/bmw/x5/158410197.html
BMW 3-Series, 2014 | 3-Series | 2014 | 185 